In [ ]:
import os
import pandas
import numpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
Panda_Data = pandas.read_table("/content/drive/MyDrive/market_basket.txt",delimiter="\t",header=0)

print(len(Panda_Data))

12935


In [ ]:
print(Panda_Data.head(10))


   ID                   Product
0   1                   Peaches
1   2             Vegetable_Oil
2   2               Frozen_Corn
3   3                     Plums
4   4               Pancake_Mix
5   5                    Cheese
6   6               Cauliflower
7   7                 2pct_Milk
8   8  98pct_Fat_Free_Hamburger
9   8              Potato_Chips


In [ ]:
print(Panda_Data.shape)

(12935, 2)


In [ ]:
colonne=Panda_Data['Product'].sort_values().unique()
ligne=Panda_Data['ID'].unique()

matrice=numpy.zeros((len(ligne),len(colonne)), int)
k=0
for i in ligne:
  df=Panda_Data[Panda_Data["ID"]==i]
  a=0
  for j in colonne:
    if df['Product'].str.contains(j).any():
      matrice[k][a]=1
    a=a+1
  k=k+1

df = pandas.DataFrame(matrice, index=ligne,columns=colonne)



In [ ]:
#5-tableau croisé avec la bib "crosstab"
CrossTab= pandas.crosstab(Panda_Data.ID,Panda_Data.Product)

#6-Affichage des 30 premières transactions et des 3 premiers produits.
print(CrossTab.iloc[:30,:3])

Product  100_Watt_Lightbulb  2pct_Milk  40_Watt_Lightbulb
ID                                                       
1                         0          0                  0
2                         0          0                  0
3                         0          0                  0
4                         0          0                  0
5                         0          0                  0
6                         0          0                  0
7                         0          1                  0
8                         0          0                  0
9                         0          0                  0
10                        0          0                  0
11                        0          0                  0
12                        0          0                  0
13                        0          0                  0
14                        0          0                  0
15                        0          0                  0
16            

In [ ]:
import itertools

def ExtractItemset(df, n):
    return list(itertools.combinations(df, n))

def CalculSupport (df,subsets,n):
  table=[]
  k=0
  for i in subsets:
    subset=df[list(i)]
    w=subset[subset.sum(axis='columns')==n].count()
    table.append(w[0])
    k=k+1
  return(table)

In [ ]:
#7-Script de l'algo apriori

nbrMax_product=2 #nbrMax_product=3 ou nbrMax_product=4 sature la mémoire
min_supp=0.025*len(ligne)

for i in range(1,nbrMax_product+1):
  itemset=ExtractItemset(colonne,i)
  Support=CalculSupport(df,itemset,i)
  ff= pandas.DataFrame(itemset)
  sup=pandas.DataFrame(Support)
  sup.columns=['Support']
  frame=[ff,sup]
  F=pandas.concat(frame,axis=1)
  C=F[F["Support"]>min_supp]  
  print("C",i)
  print(C)

C 1
                            0  Support
0          100_Watt_Lightbulb       41
1                   2pct_Milk      149
3           60_Watt_Lightbulb       51
4           75_Watt_Lightbulb       43
5    98pct_Fat_Free_Hamburger      127
..                        ...      ...
295      White_Zinfandel_Wine       42
296                Whole_Corn       42
298                Whole_Milk       48
301           flav_Fruit_Bars      125
302                  flav_Ice       51

[202 rows x 2 columns]
C 2
                 0                         1  Support
305      2pct_Milk  98pct_Fat_Free_Hamburger       52
315      2pct_Milk                    Apples       46
316      2pct_Milk                   Aspirin       47
320      2pct_Milk                   Bananas       43
330      2pct_Milk                   Bologna       39
...            ...                       ...      ...
45677      Waffles               White_Bread       63
45685      Waffles           flav_Fruit_Bars       50
45698  Wheat_B

In [ ]:
def Rule(item):
  it=item[:-1]
  for i in range(1,nbrMax_product):
    List=ExtractItemset(it,i)
    PremisList=List[:-i]
    CclList=List[i:]
    PremisSupport=CalculSupport(df,PremisList,i)
    TotalSupport=CalculSupport(df,[it],nbrMax_product)
  return(PremisList,CclList,numpy.array(TotalSupport)/numpy.array(PremisSupport))

In [ ]:
AprioriRes=[]
k=0
for i in C.index:
  k=k+1
  ligne=list(C.loc[i,:])
  Premis,Ccl,Confidence=Rule(ligne)
  print('Rule :',Premis,' -> ',Ccl,' With confidence',Confidence)

Rule : [('2pct_Milk',)]  ->  [('98pct_Fat_Free_Hamburger',)]  With confidence [0.34899329]
Rule : [('2pct_Milk',)]  ->  [('Apples',)]  With confidence [0.30872483]
Rule : [('2pct_Milk',)]  ->  [('Aspirin',)]  With confidence [0.31543624]
Rule : [('2pct_Milk',)]  ->  [('Bananas',)]  With confidence [0.2885906]
Rule : [('2pct_Milk',)]  ->  [('Bologna',)]  With confidence [0.26174497]
Rule : [('2pct_Milk',)]  ->  [('Broccoli',)]  With confidence [0.28187919]
Rule : [('2pct_Milk',)]  ->  [('Canned_Tuna',)]  With confidence [0.26845638]
Rule : [('2pct_Milk',)]  ->  [('Cantaloupe',)]  With confidence [0.24832215]
Rule : [('2pct_Milk',)]  ->  [('Chardonnay',)]  With confidence [0.29530201]
Rule : [('2pct_Milk',)]  ->  [('Cheese',)]  With confidence [0.66442953]
Rule : [('2pct_Milk',)]  ->  [('Chocolate_Bar',)]  With confidence [0.24161074]
Rule : [('2pct_Milk',)]  ->  [('Cola',)]  With confidence [0.45637584]
Rule : [('2pct_Milk',)]  ->  [('Cookies',)]  With confidence [0.46979866]
Rule : [('

In [ ]:
#extraction des itemset freq avec la bib "mlxtend" 

import mlxtend
from mlxtend.frequent_patterns import apriori

In [ ]:
freq_itemsets = apriori(CrossTab,min_support=0.025,max_len=4,use_colnames=True)


In [ ]:
type(freq_itemsets)

pandas.core.frame.DataFrame

In [ ]:
print(freq_itemsets.columns)


Index(['support', 'itemsets'], dtype='object')


In [ ]:
print(freq_itemsets.shape)


(603, 2)


In [ ]:
#8-Afficher les 15 premiers itemsets
print(freq_itemsets.head(15))


     support                    itemsets
0   0.030147        (100_Watt_Lightbulb)
1   0.109559                 (2pct_Milk)
2   0.037500         (60_Watt_Lightbulb)
3   0.031618         (75_Watt_Lightbulb)
4   0.093382  (98pct_Fat_Free_Hamburger)
5   0.031618         (AA_Cell_Batteries)
6   0.025735    (Apple_Cinnamon_Waffles)
7   0.026471               (Apple_Drink)
8   0.031618          (Apple_Fruit_Roll)
9   0.032353                 (Apple_Jam)
10  0.033088               (Apple_Jelly)
11  0.032353               (Apple_Sauce)
12  0.053676                    (Apples)
13  0.066912                   (Aspirin)
14  0.027941               (Avocado_Dip)


In [ ]:
#9-fonction fonction is_inclus() qui permet de vérifier si un sous-ensemble items est inclus dans l’ensemble x.
def is_inclus(x,items):
    return items.issubset(x)

In [ ]:
#10-Afficher les itemsets comprenant le produit ‘Aspirin’
id = numpy.where(freq_itemsets.itemsets.apply (is_inclus,items={'Aspirin'}))

print(id)

(array([ 13, 208, 249, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281,
       282, 283, 284, 285, 286, 509, 510, 511, 552, 553, 554, 555, 556]),)


In [ ]:
#11-Afficher les itemsets contenant Aspirin et Eggs
id = numpy.where(freq_itemsets.itemsets.apply (is_inclus,items={'Aspirin', 'Eggs'}))

print(id)
print(freq_itemsets.loc[id])

(array([274, 509, 552, 553]),)
      support                       itemsets
274  0.038235                (Aspirin, Eggs)
509  0.025735     (Aspirin, Eggs, 2pct_Milk)
552  0.025000  (Aspirin, Eggs, Potato_Chips)
553  0.029412   (Aspirin, White_Bread, Eggs)


In [ ]:
#Génération des règles
#fonction de calcul des règles
from mlxtend.frequent_patterns import association_rules

In [ ]:
#12-Génération des règles à partir des itemsets fréquents
regles = association_rules(freq_itemsets,metric="confidence",min_threshold=0.75)

#type de l'objet renvoyé
print(type(regles))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
#dimension
print(regles.shape)
print(regles.columns)

(50, 9)
Index(['antecedents', 'consequents', 'antecedent support',
       'consequent support', 'support', 'confidence', 'lift', 'leverage',
       'conviction'],
      dtype='object')


In [ ]:
#13- les 5 "premières" règles
print(regles.iloc[:5,:])

                 antecedents    consequents  antecedent support  \
0       (Aspirin, 2pct_Milk)  (White_Bread)            0.034559   
1     (Bananas, White_Bread)    (2pct_Milk)            0.032353   
2       (Bananas, 2pct_Milk)  (White_Bread)            0.031618   
3        (Cola, Wheat_Bread)    (2pct_Milk)            0.032353   
4  (Popcorn_Salt, 2pct_Milk)         (Eggs)            0.033088   

   consequent support   support  confidence      lift  leverage  conviction  
0            0.119118  0.027206    0.787234  6.608878  0.023089    4.140147  
1            0.109559  0.025735    0.795455  7.260525  0.022191    4.353268  
2            0.119118  0.025735    0.813953  6.833190  0.021969    4.734743  
3            0.109559  0.025735    0.795455  7.260525  0.022191    4.353268  
4            0.122794  0.027206    0.822222  6.695941  0.023143    4.934283  


In [ ]:
#règles en restreignant l'affichage à qqs colonnes
myRegles = regles.loc[:,['antecedents','consequents','lift']]
print(myRegles.shape)

(50, 3)


In [ ]:
#14-Filtrer les règles en affichant celles qui présentent un LIFT supérieur ou égal à 7.
print(myRegles[myRegles['lift'].ge(7.0)])

                                antecedents                 consequents  \
1                    (Bananas, White_Bread)                 (2pct_Milk)   
3                       (Cola, Wheat_Bread)                 (2pct_Milk)   
8                     (Onions, Wheat_Bread)                 (2pct_Milk)   
10                  (Potatoes, Wheat_Bread)                 (2pct_Milk)   
13                (Toothpaste, Wheat_Bread)                 (2pct_Milk)   
16            (Hamburger_Buns, White_Bread)  (98pct_Fat_Free_Hamburger)   
17  (98pct_Fat_Free_Hamburger, Wheat_Bread)               (White_Bread)   
29             (Hot_Dog_Buns, Sweet_Relish)                  (Hot_Dogs)   
35                 (Toilet_Paper, Potatoes)               (White_Bread)   
37               (Toothpaste, Toilet_Paper)               (White_Bread)   
40        (White_Bread, Eggs, Potato_Chips)                 (2pct_Milk)   
43          (Toothpaste, White_Bread, Eggs)                 (2pct_Milk)   
46  (Toothpaste, White_Br

In [ ]:
#15-Filtrer les règles menant au conséquent {‘2pct_milk’}
print(myRegles[myRegles['consequents'].eq({'2pct_Milk'})])


                                antecedents  consequents      lift
1                    (Bananas, White_Bread)  (2pct_Milk)  7.260525
3                       (Cola, Wheat_Bread)  (2pct_Milk)  7.260525
7                       (Eggs, Wheat_Bread)  (2pct_Milk)  6.970104
8                     (Onions, Wheat_Bread)  (2pct_Milk)  7.573897
9                (Toothpaste, Potato_Chips)  (2pct_Milk)  6.979866
10                  (Potatoes, Wheat_Bread)  (2pct_Milk)  7.053081
13                (Toothpaste, Wheat_Bread)  (2pct_Milk)  7.379694
40        (White_Bread, Eggs, Potato_Chips)  (2pct_Milk)  7.143274
43          (Toothpaste, White_Bread, Eggs)  (2pct_Milk)  7.260525
46  (Toothpaste, White_Bread, Potato_Chips)  (2pct_Milk)  7.569160
